In [1]:
import numpy as np
import xarray as xr
import copy 

In [ ]:
#Use CDO for regridding data to MPI-GE grid
#cdo -remapbil,MPI_GE_grid.txt in_file_original.nc out_file_GEgrid.nc

In [2]:
#Mask
path = '/work/uo1075/u241308/grand_ensemble_cmip6/t2max/'
file = 'nGE_Tmax_ssp245_dm_land_europe.nc'
mask = xr.open_dataset(path+file)
mask = mask.tasmax[0,0,:,:]

In [3]:
stepsize = 1
window = 15
start = int((window-1)/2)
ref_min = 1985
ref_max = 2014

In [4]:
model = 'MIROC6'
ens_number = '50'

In [5]:
# compute climatology
run = ['historical']
for n_run in run:
    # ------------------ Load data -----------------------
    path = '/work/uo1075/u241308/SMILE/'+model+'/tasmax/'
    file = model+'_Tmax_'+n_run+'_dm_ens_1-'+ens_number+'_GEgrid_europe.nc'
    with xr.open_dataset(path+file) as data:
        data = data.tasmax
        #Get rid off 29th of februarys
        data = data.sel(time=~((data.time.dt.month == 2) & (data.time.dt.day == 29)))

        data2 = np.where(mask==0,0,data) #mask ocean data
        data[:,:,:,:] = data2
        
    #flatten lon/lat
    data_flat = data.values.reshape(data.shape[0],data.shape[1],-1) 
    
    #calculate the climatology for the reference time
    if n_run == 'historical':
        climatology = []
        r_steps = 365-window+1 #number of different running windows
        for r in range(r_steps):
            sel_days = []
            for y in range(ref_min,ref_max+1):
                data_sel = data_flat[(data.time.dt.year==y)]
                data_sel = data_sel[r:r+window]
                sel_days.append(data_sel)
            
            sel_days_mean = np.array(sel_days).mean(axis=(0,1,2))
            climatology.append(sel_days_mean)
        climatology = np.array(climatology)

        climatology_complete = np.zeros((365,climatology.shape[1])) #fill values missing due to running window
        start = int((window-1)/2)
        climatology_complete[:start] = climatology[0]
        climatology_complete[-start:] = climatology[-1]
        climatology_complete[start:-start] = climatology

In [6]:
run = ['historical','ssp245','ssp585']

In [7]:
for n_run in run:
    # ------------------ Load first set -----------------------
    path = '/work/uo1075/u241308/SMILE/'+model+'/tasmax/'
    file = model+'_Tmax_'+n_run+'_dm_ens_1-'+ens_number+'_GEgrid_europe.nc'
    
    with xr.open_dataset(path+file) as data:
        data = data.tasmax
        #Get rid off 29th of februarys
        data = data.sel(time=~((data.time.dt.month == 2) & (data.time.dt.day == 29)))

        data2 = np.where(mask==0,0,data) #mask ocean data
        data[:,:,:,:] = data2
        
    #add ensemble dimension and reshape lon/lat to original format
    climatology_expanded = np.repeat(climatology_complete[:,np.newaxis,:],data.shape[1],axis=1)
    climatology_expanded = climatology_expanded.reshape(-1,data.shape[1],data.shape[2],data.shape[3])
        
    #calculate anomalies
    for y in np.unique(data.time.dt.year):
        year_sel = data[data.time.dt.year==y].values
        anomalies1 = year_sel - climatology_expanded
        data[data.time.dt.year==y] = anomalies1

# ------------------ save data -----------------------
    path = '/work/uo1075/u241308/SMILE/'+model+'/tasmax/'
    
    file = model+'_Tmax_'+n_run+'_dm_ens_1-'+ens_number+'_GEgrid_land_europe_anomaly.nc'
    data.to_netcdf(path+file)
    
    print(n_run + ' finished')

historical finished
ssp245 finished
ssp585 finished
